## GeoAI Ground-level NO2 Estimation Challenge by ITU

### Author: Hubert Kłosowski 242424

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Load data

In [ ]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))

In [ ]:
train.info()

In [ ]:
test.info()

### Extract date info

In [ ]:
def extract_date_info(dataframe):
    dataframe['Date'] = pd.to_datetime(dataframe['Date'], format='mixed')
    dataframe['DayOfWeek'] = dataframe['Date'].dt.dayofweek.astype('category')
    dataframe['Month'] = dataframe['Date'].dt.month.astype('category')
    dataframe['Year'] = dataframe['Date'].dt.year.astype('category')
    dataframe['Week'] = dataframe['Date'].dt.isocalendar().week.astype('category')
    dataframe['Season'] = get_season(dataframe['Date']).astype('category')
    return dataframe

def get_season(date_series):
    spring = ((date_series.dt.month == 3) & (date_series.dt.day >= 20)) | ((date_series.dt.month > 3) & (date_series.dt.month < 6)) | ((date_series.dt.month == 6) & (date_series.dt.day <= 20))
    summer = ((date_series.dt.month == 6) & (date_series.dt.day >= 21)) | ((date_series.dt.month > 6) & (date_series.dt.month < 9)) | ((date_series.dt.month == 9) & (date_series.dt.day <= 22))
    autumn = ((date_series.dt.month == 9) & (date_series.dt.day >= 23)) | ((date_series.dt.month > 9) & (date_series.dt.month < 12)) | ((date_series.dt.month == 12) & (date_series.dt.day <= 20))
    
    season_series = pd.Series(0, index=date_series.index)  # Domyślnie 0 dla wiosny
    season_series.loc[summer] = 1  # Lato
    season_series.loc[autumn] = 2  # Jesień
    season_series.loc[~(spring | summer | autumn)] = 3  # Zima
    
    return season_series

train = extract_date_info(train)
test = extract_date_info(test)

### Target values in day of week

In [ ]:
import matplotlib.patches as mpatches


def compare_plots(column_name):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(25, 15))
    
    sns.barplot(data=train, x=column_name, y='GT_NO2', ax=ax[0], color='blue')
    sns.barplot(data=train, x=column_name, y='NO2_trop', ax=ax[1], color='blue')
    sns.barplot(data=test, x=column_name, y='NO2_trop', ax=ax[1], color='orange')
    
    train_patch = mpatches.Patch(color='blue', label='Train')
    test_patch = mpatches.Patch(color='orange', label='Test')
    
    ax[1].legend(handles=[train_patch, test_patch], title='NO2_trop', loc='upper right')
    plt.show()
    

compare_plots('DayOfWeek')

### Target values in months

In [ ]:
compare_plots('Month')

### Target values in years

In [ ]:
compare_plots('Year')

### Target values in weeks

In [ ]:
compare_plots('Week')

### Target values in seasons

In [ ]:
compare_plots('Season')

### Map of id's in train dataset

In [ ]:
import folium


my_map = folium.Map(
    location=(train['LAT'].mean(), train['LON'].mean()),
    zoom_start=7,
)

### Train Locations

In [ ]:
unique_train_locations = train.groupby(['LAT', 'LON'])['GT_NO2'].mean().reset_index()
unique_train_locations = pd.merge(unique_train_locations, pd.read_csv(os.path.join('data', 'train_elevation.csv'), index_col=0))
nans = train.loc[train['GT_NO2'].isna() == True, ['LAT', 'LON']].value_counts().to_frame().reset_index()
unique_train_locations = pd.merge(unique_train_locations, nans, on=['LAT', 'LON'], how='outer')
unique_train_locations.rename({'count': 'Nans'}, axis=1, inplace=True)
unique_train_locations['Nans'] = unique_train_locations['Nans'].apply(lambda x: round(x / 10.96, 2))

unique_train_locations

In [ ]:
layer_train_map = folium.FeatureGroup(name='Train Locations', show=False)
for index, row in unique_train_locations.iterrows():
    popup_html = f"""
    <div style="font-family: Arial; font-size: 14px;">
        <strong>Mean GT_NO2 level:</strong> {row['GT_NO2']:.2f}<br>
        <strong>Latitude:</strong> {row['LAT']}<br>
        <strong>Longitude:</strong> {row['LON']}<br>
        <strong>Percent of nans:</strong> {row['Nans']} %<br>
        <strong>Elevation:</strong> {row['Elevation']}
    </div>
    """
    popup = folium.Popup(popup_html, max_width=300)
    if row['Nans'] > 10:
        folium.Marker(
            location=[row['LAT'], row['LON']],
            icon=folium.Icon(color='green', icon_color='black', icon='home'),
            popup=popup,
        ).add_to(layer_train_map)
    else:
        folium.Marker(
            location=[row['LAT'], row['LON']],
            icon=folium.Icon(color='green', icon='home'),
            popup=popup,
        ).add_to(layer_train_map)
    
layer_train_map.add_to(my_map)

### Test Locations

In [ ]:
unique_test_locations = pd.read_csv(os.path.join('data', 'test_elevation.csv'), index_col=0)

layer_test_map = folium.FeatureGroup(name='Test Locations', show=False)
for index, row in unique_test_locations.iterrows():
    popup_html = f"""
    <div style="font-family: Arial; font-size: 14px;">
        <strong>Latitude:</strong> {row['LAT']}<br>
        <strong>Longitude:</strong> {row['LON']}<br>
        <strong>Elevation:</strong> {row['Elevation']}
    </div>
    """
    popup = folium.Popup(popup_html, max_width=300)
    folium.Marker(
        location=[row['LAT'], row['LON']],
        icon=folium.Icon(color='red', icon='home'),
        popup=popup,
    ).add_to(layer_test_map)
    
layer_test_map.add_to(my_map)

In [ ]:
folium.LayerControl().add_to(my_map)
my_map.save('my_map.html')

### Do sth with NaNs in GT_NO2

- drop
- interpolate but only for locations that have 10% NaNs

In [ ]:
# ok_nans = unique_train_locations.loc[unique_train_locations['Nans'] <= 5.0, ['LAT', 'LON']]
# train = pd.merge(train, ok_nans, on=['LAT', 'LON'], how='inner')
# groups = train.groupby(['LAT', 'LON'])
# 
# for name, group in groups:
#     group.set_index('Date', inplace=True)
#     group['GT_NO2'] = group['GT_NO2'].interpolate(method='time')
#     
# train = pd.concat([group for name, group in groups])

train.dropna(subset=['GT_NO2'], inplace=True)
train.reset_index(drop=True, inplace=True)

train.head()

### Distribution of values

In [ ]:
sns.histplot(data=train, x='GT_NO2')

### Prepare data

In [ ]:
test_ids = test['ID_Zindi']
train.drop(columns=['ID', 'ID_Zindi', 'Date'], axis=1, inplace=True)
test.drop(columns=['ID', 'ID_Zindi', 'Date'], axis=1, inplace=True)

### Get elevation from LAT, LON

In [ ]:
unique_train_locations.drop(columns=['GT_NO2', 'Nans'], inplace=True)

train = pd.merge(train, unique_train_locations, on=['LAT', 'LON'], how='left')
test = pd.merge(test, unique_test_locations, on=['LAT', 'LON'], how='left')

In [ ]:
train.head()

### Check ranges of values

In [ ]:
train.describe()

### Make the same ranges for test and train

In [ ]:
def cut_ranges():
    for column in test.select_dtypes(exclude=['category']).columns[2:]:
        test_min = test[column].min()
        test_max = test[column].max()
        to_drop = train[(train[column] < test_min) | (train[column] > test_max)].index
        train.drop(index=to_drop, inplace=True)


cut_ranges()

train.describe()

### Correlation Matrix for train

In [ ]:
def compare_correlation(df):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(28, 13))
    
    sns.heatmap(df.corr().round(2), annot=True, cmap='Greys', ax=ax[0])
    sns.heatmap(df.corr(method='spearman').round(2), annot=True, cmap='Greys', ax=ax[1])
    plt.show()

compare_correlation(train)

### Correlation Matrix for test

In [ ]:
compare_correlation(test)

### Scatter plots of columns from original dataset

In [ ]:
def plot_scatter():
    columns = test.select_dtypes(exclude=['category']).columns[2:]
    fig, ax = plt.subplots(nrows=(len(columns) + 2) // 3, ncols=3, figsize=(25, 15))
    for i, col in enumerate(columns):
        x_cord, y_cord = divmod(i, 3)
        sns.scatterplot(data=train, x=col, y='GT_NO2', ax=ax[x_cord, y_cord], s=2)
        ax[x_cord, y_cord].set_title(f'{col} and GT_NO2')
        ax[x_cord, y_cord].set_xlabel(col)
        ax[x_cord, y_cord].set_ylabel('GT_NO2')


plot_scatter()

### Identify outliers in target column using zscore

In [ ]:
from scipy.stats import zscore


def show_outliers(data, up, down, num):
    outliers = zscore(data)
    
    quantiles = pd.DataFrame(list(zip(np.linspace(up, down, num), [np.quantile(outliers, el) for el in np.linspace(up, down, num)], [np.quantile(data, el) for el in np.linspace(up, down, num)])), columns=['quantile', 'zscore', data.name])
    
    return quantiles

show_outliers(train['GT_NO2'], 0.98, 1, 21)

In [ ]:
show_outliers(train['GT_NO2'], 0.0, 0.1, 21)

### Deleting outliers in top 1.4% percentile

In [ ]:
def detect_outliers(data, down, up):
    outliers = zscore(data)
    q1, q2 = np.quantile(outliers, down), np.quantile(outliers, up)
    return [i for i, el in enumerate(outliers) if el > q2]

def delete_outliers(indexes, df):
    df.reset_index(drop=True, inplace=True)
    df.drop(indexes, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

indexes_to_drop = detect_outliers(train['GT_NO2'], 0.0, 0.986)
train = delete_outliers(indexes_to_drop, train)

train.info()

### Save data

In [ ]:
train.to_csv(os.path.join('prepared', 'train.csv'), index=False)
test.to_csv(os.path.join('prepared', 'test.csv'), index=False)